In [ ]:
!pip install torch

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split 
from collections import Counter
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook as tqdm
import torch.utils.data
from torch.utils.data import Dataset, DataLoader
import matplotlib.font_manager
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split

In [ ]:
# Ask user for the dataset1 path
path1 = input("Enter the path to the dataset-1: ")

# Load the dataset
try:
    data = pd.read_csv(path1)
except Exception as e:
    print(f"Error loading the dataset: {e}: please restart the program")
    exit()
    #C:\Users\JILCHALELISAADEBA\test_set-2.csv

In [ ]:
#remove the index column
data.drop(columns=data.columns[0], axis=1,  inplace=True)

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
def normalization(dat,column):
  for i in column:
    arr = dat[i]
    arr = np.array(arr)
    dat[i] = scaler.fit_transform(arr.reshape(len(arr),1))
  return dat

In [ ]:
scaled_data = normalization(data.copy(),data.columns)

In [ ]:
# Shuffle the test DataFrame
test_df = data.sample(frac=1).reset_index(drop=True)

In [ ]:
y_te = test_df['label']
test_df.drop('label', axis=1, inplace=True)
test_df.drop('id', axis=1, inplace=True)

In [ ]:
#convert the dataset to numpy array, before converting to torch tensor
x_te_np = test_df.to_numpy().astype(np. float32) 
y_te_np = y_te.to_numpy().astype(np. float32) 

In [ ]:
# Convert the numpy arrays to PyTorch tensors
X_te_tensor = torch.tensor(x_te_np)
y_te_tensor = torch.tensor(y_te_np)

In [ ]:
# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [ ]:
# Create DataLoader for training and testing
batch_size = 32
test_dataset = CustomDataset(X_te_tensor, y_te_tensor.reshape(-1, 1))
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
#class abstruction of LSTM_VAE model
class LSTM_AE(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.enc1 = nn.LSTM(
          input_size=240,
          hidden_size=128,
          num_layers=1,
          batch_first=True,
          dropout = 0.6
        )
        self.enc2 = nn.LSTM(
          input_size=128,
          hidden_size=4,
          num_layers=1,
          batch_first=True,
          dropout = 0.6
        )
        
        self.dec1 = nn.LSTM(
          input_size=4,
          hidden_size=128,
          num_layers=1,
          batch_first=True,
          dropout = 0.6
        )
        self.dec2 = nn.LSTM(
          input_size=128,
          hidden_size=240,
          num_layers=1,
          batch_first=True,
          dropout = 0.6
        )
        
    def forward(self, x):
        x, (_, _) = self.enc1(x)
        x, (hidden_n, _) = self.enc2(x)
        x, (hidden_n, cell_n) = self.dec1(x)
        x, (hidden_n, cell_n) = self.dec2(x)
        return  x
    
    def run_encoder(self, x):
        x = self.enc1(x)
        x = self.enc2(x)
        latent  = x
        return latent

In [ ]:
# Class abstraction of FC classifier 
class SimpleClassifier(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
        super(SimpleClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        #self.fc4 = nn.Linear(hidden_size3, hidden_size4)
        self.fc5 = nn.Linear(hidden_size3, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        #x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x

In [ ]:
# Class abstraction of Conv_VAE model
class Conv_VAE(nn.Module):
    def __init__(self, kernel_size = 2,
                init_channels = 8, 
                data_channels = 1, 
                latent_dim = 32): 
        super(ConvVAE6, self).__init__()
 
        # encoder (in_channels=1, out_channels=1, kernel_size=3, stride=1)
        self.enc1 = nn.Conv1d(
            in_channels=1, out_channels=init_channels, kernel_size=kernel_size,stride=1, padding=0)
        self.enc2 = nn.Conv1d(
            in_channels=init_channels, out_channels=init_channels*2, kernel_size=kernel_size,stride=1, padding=1)
        self.enc3 = nn.Conv1d(
            in_channels=init_channels*2, out_channels=init_channels*4, kernel_size=kernel_size,stride=2, padding=1)
        self.enc4 = nn.Conv1d(
            in_channels=init_channels*4, out_channels=64, kernel_size=kernel_size,stride=2, padding=0)
        
        # FC for teh hidden representations
        self.fc1 = nn.Linear(61, 128)
        self.fc_mu = nn.Linear(128, latent_dim)
        self.fc_log_var = nn.Linear(128, latent_dim)
        self.fc2 = nn.Linear(latent_dim, 64)
        # decoder 
        self.dec1 = nn.ConvTranspose1d(
            in_channels=64, out_channels=init_channels*8, kernel_size=kernel_size,stride=1, padding=0)
        self.dec2 = nn.ConvTranspose1d(
            in_channels=init_channels*8, out_channels=init_channels*4, kernel_size=kernel_size,stride=2, padding=2)
        self.dec3 = nn.ConvTranspose1d(
            in_channels=init_channels*4, out_channels=init_channels*2, kernel_size=kernel_size,stride=2, padding=3)
        self.dec4 = nn.ConvTranspose1d(
            in_channels=init_channels*2, out_channels=data_channels, kernel_size=kernel_size,stride=1, padding=2)
    def reparameterize(self, mu, log_var):
        """
        :param mu: mean from the encoder's latent space
        :param log_var: log variance from the encoder's latent space
        """
        #standard deviation
        std = torch.exp(0.5*log_var) 
        #`randn_like` as we need the same size
        eps = torch.randn_like(std) 
        #sampling
        sample = mu + (eps * std) 
        return sample
 
    def forward(self, x):
        # encoding
        x = torch.nn.functional.relu(self.enc1(x))
        x = torch.nn.functional.relu(self.enc2(x))
        x = torch.nn.functional.relu(self.enc3(x))
        x = torch.nn.functional.relu(self.enc4(x))
    
        batch, _, _ = x.shape
        #x = F.adaptive_avg_pool1d(x, 64).reshape(batch, -1)
        #print(x.shape)
        hidden = self.fc1(x)
        # get `mu` and `log_var`
        mu = self.fc_mu(hidden)
        log_var = self.fc_log_var(hidden)
        
        # get the latent vector through reparameterization
        z = self.reparameterize(mu, log_var)
        z = self.fc2(z)
        z = z.view(-1, 64)
        #print(z.shape[0])
        z = z.view(-1, z.shape[0]//64, 64)
        z = torch.moveaxis(z, 0, 1)
        
        # decoding
        x = torch.nn.functional.relu(self.dec1(z))
        x = torch.nn.functional.relu(self.dec2(x))
        x = torch.nn.functional.relu(self.dec3(x))
        reconstruction = torch.sigmoid(self.dec4(x))
        reconstruction = reconstruction[:, :, :59]
        return reconstruction, mu, log_var

In [ ]:
# Ask user for the model path
path3 = input("Enter the path to the model: ")

# Load the dataset
try:
    model = torch.load(path3, map_location=torch.device('cpu'))
except Exception as e:
    print(f"Error loading the model: {e}: please restart the program")
    exit()
    #C:\Users\JILCHALELISAADEBA\ISAA-IITP_trained_model.pth

In [ ]:
# Evaluate on the test set
model.eval()
model.to('cpu')
correct = 0
total = 0
y_true = []
y_pred = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inps = inputs
        labs = labels
        outs = model(inps)
        predicted = (torch.sigmoid(outs) > 0.5).float()
        
        y_true.extend(labs.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())
        
        total += labs.size(0)
        correct += (predicted == labs).sum().item()

print('Accuracy: ', accuracy_score(y_true, y_pred))
print('F1 score: ', f1_score(y_true, y_pred))
print('Recall: ', recall_score(y_true, y_pred))
print('Precision: ', precision_score(y_true, y_pred))
print('\n clasification report:\n ', classification_report(y_true, y_pred))
print('\n confussion matrix:\n ',confusion_matrix(y_true, y_pred))